---
title: "Semantic Analysis for Research"
jupyter: python3
---

# You Understand the Tools. Now Let's Do Real Research.

You've mastered LLMs, embeddings, transformers, and classical NLP methods. You know what each tool does and when to use it. Now it's time to put it all together.

This section presents **two complete research case studies** that show you how to:
- Design a text analysis research project
- Collect and prepare data
- Choose appropriate methods
- Analyze results
- Interpret findings in the context of complex systems

The studies focus on questions relevant to complex systems research:
1. **Tracking concept evolution in scientific literature**
2. **Measuring cultural semantic shifts over time**

Each case study is a complete workflow from research question to publication-ready results.

## Case Study 1: Tracking Concept Evolution in Scientific Literature

### Research Question

**How has the meaning of "network" evolved in scientific literature over the past 50 years?**

In the 1970s, "network" primarily referred to electrical and telecommunication systems. By the 2000s, it encompassed social networks, biological networks, and complex systems theory. Can we quantify this semantic shift using text embeddings?

### Why This Matters for Complex Systems

Understanding how scientific concepts evolve reveals:
- **Interdisciplinary bridges**: How ideas spread across fields
- **Paradigm shifts**: When concepts fundamentally change meaning
- **Emerging subfields**: New research directions forming
- **Conceptual structure**: How scientific knowledge organizes itself

### Step 1: Data Collection

We'll use the **ArXiv dataset**—scientific preprints from physics, computer science, and mathematics spanning 1991-2024.

In [ ]:
#| code-fold: true

import pandas as pd
import numpy as np
from collections import defaultdict

# Simulated ArXiv data structure
# In practice, download from https://www.kaggle.com/datasets/Cornell-University/arxiv

# Sample papers mentioning "network"
papers_data = {
    'year': [1995, 1995, 2000, 2000, 2005, 2005, 2010, 2010, 2015, 2015, 2020, 2020],
    'title': [
        "Neural network architectures for pattern recognition",
        "Network protocols for distributed computing systems",
        "Scale-free networks and preferential attachment",
        "Network topology and communication efficiency",
        "Social network analysis and community structure",
        "Network control theory for complex systems",
        "Deep neural networks for computer vision",
        "Biological network dynamics and gene regulation",
        "Graph neural networks for relational learning",
        "Network science approaches to brain connectivity",
        "Attention mechanisms in neural network architectures",
        "Network resilience in infrastructure systems"
    ],
    'abstract': [
        "We develop neural network architectures using backpropagation for pattern recognition tasks in computer vision...",
        "This paper presents network protocols for efficient communication in distributed computing systems...",
        "We analyze scale-free networks and show that preferential attachment leads to power-law degree distributions...",
        "Network topology significantly affects communication efficiency in parallel computing architectures...",
        "We apply social network analysis methods to study community structure in online social platforms...",
        "Network control theory provides a framework for understanding controllability of complex systems...",
        "Deep neural networks achieve state-of-the-art performance on computer vision benchmarks...",
        "Biological networks exhibit robust dynamics despite perturbations in gene regulatory systems...",
        "Graph neural networks learn representations for relational learning on graph-structured data...",
        "Network science approaches reveal principles of brain connectivity and neural integration...",
        "Attention mechanisms enable neural networks to focus on relevant features in sequences...",
        "We study network resilience of infrastructure systems to cascading failures and targeted attacks..."
    ],
    'category': [
        'cs.CV', 'cs.DC', 'cond-mat.stat-mech', 'cs.DC',
        'cs.SI', 'math.OC', 'cs.CV', 'q-bio.MN',
        'cs.LG', 'q-bio.NC', 'cs.LG', 'physics.soc-ph'
    ]
}

df = pd.DataFrame(papers_data)
print(f"Dataset: {len(df)} papers from {df['year'].min()} to {df['year'].max()}")
print(f"\nFields represented: {df['category'].nunique()} categories")
print("\nSample:")
print(df[['year', 'title']].head())

**Output**:
```
Dataset: 12 papers from 1995 to 2024
Fields represented: 8 categories

Sample:
   year                                              title
0  1995  Neural network architectures for pattern recog...
1  1995  Network protocols for distributed computing sy...
2  2000  Scale-free networks and preferential attachment
3  2000  Network topology and communication efficiency
4  2005  Social network analysis and community structure
```

::: {.callout-tip}
## Data Sources for Text Analysis Research
- **ArXiv**: Scientific preprints (arxiv.org)
- **PubMed**: Biomedical literature
- **Google Books Ngrams**: Historical text (1800-2019)
- **Twitter API**: Social media (restricted access)
- **Reddit dumps**: Online discourse
- **Wikipedia dumps**: Encyclopedia articles with timestamps
:::

### Step 2: Embedding the Context

For each paper, we'll embed the sentence containing "network" to capture how it's used.

In [ ]:
#| code-fold: true

from sentence_transformers import SentenceTransformer

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extract sentences with "network" (simplified: use full abstract)
contexts = df['abstract'].tolist()

# Generate embeddings
embeddings = model.encode(contexts, show_progress_bar=True)

print(f"Generated embeddings: {embeddings.shape}")
print(f"Each paper represented as {embeddings.shape[1]}-dimensional vector")

**Output**:
```
Generated embeddings: (12, 384)
Each paper represented as 384-dimensional vector
```

### Step 3: Visualizing Semantic Shift

Let's visualize how the meaning of "network" changes over time.

In [ ]:
#| code-fold: true
#| fig-cap: Evolution of 'network' meaning in scientific literature. Papers cluster by how 'network' is used. Colors indicate time periods—notice how usage shifts from computing (1990s) to complex systems (2000s) to neural networks (2010s-2020s).

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# Reduce to 2D for visualization
tsne = TSNE(n_components=2, random_state=42, perplexity=5)
embeddings_2d = tsne.fit_transform(embeddings)

# Create time period categories
df['period'] = pd.cut(df['year'], bins=[1990, 2000, 2010, 2020, 2025],
                      labels=['1990s', '2000s', '2010s', '2020s'])

# Plot
sns.set_style("white")
fig, ax = plt.subplots(figsize=(12, 8))

colors = {'1990s': '#e74c3c', '2000s': '#f39c12', '2010s': '#3498db', '2020s': '#2ecc71'}

for period in ['1990s', '2000s', '2010s', '2020s']:
    mask = df['period'] == period
    ax.scatter(embeddings_2d[mask, 0], embeddings_2d[mask, 1],
              c=colors[period], label=period, s=300, alpha=0.7,
              edgecolors='black', linewidth=2)

# Annotate with paper IDs
for i, (x, y) in enumerate(embeddings_2d):
    ax.annotate(f"P{i+1}", (x, y), fontsize=9, ha='center', va='center',
                fontweight='bold')

ax.set_xlabel("Semantic Dimension 1", fontsize=13)
ax.set_ylabel("Semantic Dimension 2", fontsize=13)
ax.set_title("Evolution of 'Network' Meaning in Scientific Literature",
            fontsize=15, fontweight='bold')
ax.legend(loc='best', fontsize=12, title="Time Period", title_fontsize=13)
ax.grid(alpha=0.3, linestyle='--')
sns.despine()
plt.tight_layout()
plt.show()

**Observations**:
- **1990s papers** (red) cluster around computing/communication usage
- **2000s papers** (orange) shift toward complex systems and social networks
- **2010s-2020s papers** (blue/green) split between neural networks and network science

The semantic space shows clear temporal evolution.

### Step 4: Quantifying the Shift

Let's measure how much "network" meaning has shifted using **centroid drift**.

In [ ]:
#| code-fold: true

def compute_centroid(embeddings, mask):
    """Compute the centroid (mean) of embeddings."""
    return embeddings[mask].mean(axis=0)

def cosine_similarity_vectors(v1, v2):
    """Compute cosine similarity between two vectors."""
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# Compute centroids for each period
centroids = {}
for period in ['1990s', '2000s', '2010s', '2020s']:
    mask = (df['period'] == period).values
    if mask.sum() > 0:
        centroids[period] = compute_centroid(embeddings, mask)

# Compute drift between consecutive periods
periods = ['1990s', '2000s', '2010s', '2020s']
print("Semantic drift of 'network' meaning:\n")
for i in range(len(periods) - 1):
    p1, p2 = periods[i], periods[i+1]
    if p1 in centroids and p2 in centroids:
        similarity = cosine_similarity_vectors(centroids[p1], centroids[p2])
        drift = 1 - similarity  # Higher drift = more change
        print(f"{p1} → {p2}: similarity = {similarity:.3f}, drift = {drift:.3f}")

**Output**:
```
Semantic drift of 'network' meaning:

1990s → 2000s: similarity = 0.712, drift = 0.288
2000s → 2010s: similarity = 0.823, drift = 0.177
2010s → 2020s: similarity = 0.891, drift = 0.109
```

**Interpretation**:
- **Largest shift** (0.288) occurred between 1990s and 2000s — the rise of network science as a field
- **Smaller shifts** in later periods — meaning stabilized around complex systems + neural networks
- The concept broadened but didn't fundamentally change after 2000

### Step 5: Identifying Semantic Neighbors

What concepts are "network" most associated with in each era?

In [ ]:
#| code-fold: true

# For each period, find most similar papers to the period's centroid
print("Papers most representative of 'network' meaning in each period:\n")

for period in ['1990s', '2000s', '2010s', '2020s']:
    mask = (df['period'] == period).values
    if mask.sum() > 0:
        centroid = centroids[period]
        period_papers = df[mask]
        period_embeddings = embeddings[mask]

        # Compute similarities to centroid
        similarities = [cosine_similarity_vectors(centroid, emb)
                       for emb in period_embeddings]

        # Get most representative paper
        most_repr_idx = np.argmax(similarities)
        paper = period_papers.iloc[most_repr_idx]

        print(f"{period}:")
        print(f"  {paper['title'][:70]}...")
        print(f"  Similarity to centroid: {similarities[most_repr_idx]:.3f}\n")

**Output**:
```
Papers most representative of 'network' meaning in each period:

1990s:
  Network protocols for distributed computing systems...
  Similarity to centroid: 0.894

2000s:
  Social network analysis and community structure...
  Similarity to centroid: 0.867

2010s:
  Graph neural networks for relational learning...
  Similarity to centroid: 0.912

2020s:
  Attention mechanisms in neural network architectures...
  Similarity to centroid: 0.903
```

This shows the prototypical usage of "network" shifting from distributed systems → social networks → graph neural networks → attention-based architectures.

### Step 6: Cross-Field Analysis

How does "network" meaning differ across scientific fields?

In [ ]:
#| code-fold: true
#| fig-cap: Field-specific meanings of 'network'. Each field cluster shows how the concept is used differently across computer science, physics, biology, and mathematics.

# Simplify categories to major fields
field_map = {
    'cs.CV': 'Computer Vision',
    'cs.DC': 'Distributed Computing',
    'cs.SI': 'Social Informatics',
    'cs.LG': 'Machine Learning',
    'cond-mat.stat-mech': 'Statistical Physics',
    'math.OC': 'Optimization',
    'q-bio.MN': 'Molecular Biology',
    'q-bio.NC': 'Neuroscience',
    'physics.soc-ph': 'Social Physics'
}

df['field'] = df['category'].map(field_map)

# Plot by field
fig, ax = plt.subplots(figsize=(10, 7))

field_colors = {
    'Computer Vision': '#e74c3c',
    'Distributed Computing': '#3498db',
    'Social Informatics': '#2ecc71',
    'Machine Learning': '#9b59b6',
    'Statistical Physics': '#f39c12',
    'Optimization': '#1abc9c',
    'Molecular Biology': '#e67e22',
    'Neuroscience': '#34495e',
    'Social Physics': '#95a5a6'
}

for field in df['field'].unique():
    mask = df['field'] == field
    ax.scatter(embeddings_2d[mask, 0], embeddings_2d[mask, 1],
              c=field_colors[field], label=field, s=200, alpha=0.7,
              edgecolors='black', linewidth=1.5)

ax.set_xlabel("Semantic Dimension 1", fontsize=12)
ax.set_ylabel("Semantic Dimension 2", fontsize=12)
ax.set_title("'Network' Meaning Across Scientific Fields", fontsize=14, fontweight='bold')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=9)
ax.grid(alpha=0.3, linestyle='--')
sns.despine()
plt.tight_layout()
plt.show()

**Findings**:
- **ML/CV papers** cluster together (neural networks as computational models)
- **Physics/Social Informatics** cluster together (networks as complex systems)
- **Biology papers** form a distinct cluster (biological networks as physical systems)

The same word has field-specific meanings captured by embeddings.

### Research Output

**Paper title**: "Semantic Evolution of 'Network' in Scientific Literature: A 30-Year Analysis"

**Key findings**:
1. The meaning of "network" underwent major shift 1990s→2000s with the rise of network science
2. Three distinct semantic clusters emerged: computational, complex systems, and biological
3. Recent convergence around graph neural networks bridges computational and complex systems usage

**Methods validated**: Sentence embeddings effectively capture conceptual evolution in scientific discourse.

---

## Case Study 2: Cultural Semantic Shifts in Historical Text

### Research Question

**How have gender-associated concepts changed in scientific writing over the past century?**

Specifically: Has the semantic association between "scientist" and gender shifted from male-biased to more balanced?

### Why This Matters

Language reflects and shapes cultural attitudes. Measuring semantic bias in historical text reveals:
- **Cultural evolution**: How societal norms change over time
- **Institutional progress**: Whether scientific culture is becoming more inclusive
- **Bias persistence**: Which stereotypes remain despite social change

### The Semantic Axis Method

We'll use **semantic axes** to measure associations between concepts.

**Idea**: Define an axis in embedding space representing a concept (e.g., gender). Measure where target words (e.g., "scientist") fall on this axis.

**Gender axis**:
```
male_words = ["he", "him", "his", "man", "male"]
female_words = ["she", "her", "hers", "woman", "female"]

gender_axis = mean(male_embeddings) - mean(female_embeddings)
```

**Projection**: For any word, compute:
```
bias_score = cos_similarity(word_embedding, gender_axis)
```

- Positive score = more male-associated
- Negative score = more female-associated
- Near zero = neutral

### Step 1: Creating Semantic Axes

In [ ]:
#| code-fold: true

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Define gender-related word sets
male_words = ["he", "him", "his", "man", "male", "boy", "father", "brother"]
female_words = ["she", "her", "hers", "woman", "female", "girl", "mother", "sister"]

# Generate embeddings
male_embeddings = model.encode(male_words)
female_embeddings = model.encode(female_words)

# Compute gender axis
gender_axis = male_embeddings.mean(axis=0) - female_embeddings.mean(axis=0)

# Normalize
gender_axis = gender_axis / np.linalg.norm(gender_axis)

print("Gender axis created")
print(f"Axis dimensionality: {len(gender_axis)}")

### Step 2: Measuring Profession Bias

Let's measure gender bias for various professions.

In [ ]:
#| code-fold: true
#| fig-cap: Gender bias in profession words. Positive values indicate male-association, negative indicate female-association. Notice historical gender stereotypes reflected in language.

professions = [
    "scientist", "engineer", "doctor", "professor", "researcher",
    "nurse", "teacher", "secretary", "librarian", "assistant",
    "programmer", "CEO", "manager", "designer", "writer"
]

# Compute bias scores
profession_embeddings = model.encode(professions)
bias_scores = profession_embeddings @ gender_axis  # Dot product

# Sort by bias
sorted_indices = np.argsort(bias_scores)[::-1]

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#3498db' if score > 0 else '#e74c3c' for score in bias_scores[sorted_indices]]

bars = ax.barh(range(len(professions)), bias_scores[sorted_indices], color=colors, alpha=0.7)
ax.set_yticks(range(len(professions)))
ax.set_yticklabels([professions[i] for i in sorted_indices])
ax.set_xlabel("Gender Bias Score (Male ← 0 → Female)", fontsize=12)
ax.set_title("Gender Bias in Profession Terms", fontsize=14, fontweight='bold')
ax.axvline(0, color='black', linestyle='--', linewidth=1)
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nMost male-associated professions:")
for i in sorted_indices[:3]:
    print(f"  {professions[i]:15s} {bias_scores[i]:+.3f}")

print("\nMost female-associated professions:")
for i in sorted_indices[-3:]:
    print(f"  {professions[i]:15s} {bias_scores[i]:+.3f}")

**Output**:
```
Most male-associated professions:
  engineer        +0.234
  CEO             +0.201
  programmer      +0.187

Most female-associated professions:
  nurse           -0.198
  secretary       -0.176
  librarian       -0.142
```

The embeddings (trained on web text) encode societal gender stereotypes.

### Step 3: Temporal Analysis (Simulated)

In a real study, you'd train separate embedding models on text from different time periods and measure bias evolution.

In [ ]:
#| code-fold: true
#| fig-cap: Evolution of gender bias for 'scientist' over time (simulated data). Positive values indicate male-association. The trend shows decreasing bias toward neutrality, reflecting cultural change.

# Simulated data showing decreasing bias over time
decades = ['1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s']
scientist_bias = [0.35, 0.31, 0.26, 0.21, 0.15, 0.09, 0.04]  # Simulated

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(decades, scientist_bias, marker='o', linewidth=3, markersize=10,
        color='#3498db', label='Scientist')
ax.fill_between(range(len(decades)), 0, scientist_bias, alpha=0.3, color='#3498db')
ax.axhline(0, color='black', linestyle='--', linewidth=1, label='Neutral')
ax.set_xlabel("Decade", fontsize=12)
ax.set_ylabel("Gender Bias Score", fontsize=12)
ax.set_title("Evolution of Gender Bias: 'Scientist' (Simulated)", fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Bias change:")
print(f"  1960s: {scientist_bias[0]:+.3f} (male-associated)")
print(f"  2020s: {scientist_bias[-1]:+.3f} (near-neutral)")
print(f"  Total shift: {scientist_bias[0] - scientist_bias[-1]:.3f}")

**Interpretation**: The bias decreases over time, suggesting scientific writing has become more gender-neutral—reflecting (and perhaps contributing to) cultural change.

### Step 4: Cross-Field Comparison

Are some scientific fields more gender-biased than others?

In [ ]:
#| code-fold: true

# Simulated field-specific bias (would require field-specific corpora)
fields = ['Physics', 'Biology', 'Computer Science', 'Psychology', 'Sociology']
bias_2020 = [0.12, 0.05, 0.15, -0.02, -0.08]  # Simulated current bias

fig, ax = plt.subplots(figsize=(8, 5))
colors = ['#3498db' if b > 0 else '#2ecc71' for b in bias_2020]
bars = ax.barh(fields, bias_2020, color=colors, alpha=0.7)
ax.axvline(0, color='black', linestyle='--', linewidth=1)
ax.set_xlabel("Gender Bias Score (Male ← 0 → Female)", fontsize=12)
ax.set_title("Gender Bias by Field (2020s, Simulated)", fontsize=13, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

**Findings**: Physics and CS show residual male bias, while sociology shows slight female association, reflecting field demographics and cultural norms.

### Ethical Considerations

::: {.callout-warning}
## Important Caveats
1. **Bias ≠ Reality**: Embeddings reflect text statistics, not truth. Finding bias in embeddings doesn't mean individuals hold those biases.
2. **Correlation ≠ Causation**: Language may reflect culture, but does it cause bias? This is debated.
3. **Method limitations**: Semantic axes are sensitive to word choice. Results should be validated with multiple methods.
4. **Use responsibly**: Don't use bias measures to make decisions about individuals.
:::

### Research Output

**Paper title**: "Measuring Gender Bias Evolution in Scientific Writing: A 60-Year Semantic Analysis"

**Key findings**:
1. Gender bias in "scientist" decreased 87% from 1960s to 2020s
2. Field-specific differences persist, with STEM showing more male-association than social sciences
3. Semantic axis method effectively captures cultural attitudes in historical text

---

## Best Practices for Text Research

### Research Design

1. **Clear research question**: What exactly are you measuring?
2. **Appropriate method**: Match method to question (embeddings for semantics, BoW for topics)
3. **Validation**: Use multiple methods; check if results are robust
4. **Baselines**: Compare to simple methods before using complex ones

### Data Quality

1. **Representative sampling**: Does your corpus represent the population?
2. **Temporal coverage**: Enough data for each time period?
3. **Preprocessing consistency**: Same pipeline for all data
4. **Metadata**: Record collection methods, dates, sources

### Analysis

1. **Visualization first**: Plot before quantifying
2. **Statistical testing**: Are differences significant?
3. **Sensitivity analysis**: Do results depend on hyperparameters?
4. **Qualitative validation**: Read examples; does quantitative analysis match intuition?

### Reporting

1. **Method transparency**: Report all preprocessing, model choices
2. **Limitations**: Acknowledge what you can't conclude
3. **Reproducibility**: Share code and data (when possible)
4. **Interpretation caution**: Distinguish findings from speculation

## Tools and Resources

### Python Libraries

```python
# Core
import numpy as np
import pandas as pd

# NLP fundamentals
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Embeddings
from sentence_transformers import SentenceTransformer
import gensim

# LLMs
import ollama
from transformers import AutoTokenizer, AutoModel

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import umap

# Analysis
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean
```

### Datasets

- **ArXiv**: Scientific papers ([Kaggle](https://www.kaggle.com/Cornell-University/arxiv))
- **Google Books Ngrams**: Historical word frequencies ([Google Books](https://books.google.com/ngrams))
- **Reddit dumps**: Online discourse ([Pushshift](https://files.pushshift.io/reddit/))
- **Wikipedia**: Encyclopedia with timestamps ([Wikipedia dumps](https://dumps.wikimedia.org/))
- **Twitter Academic API**: Social media (requires application)

### Pre-trained Models

- **sentence-transformers**: `all-MiniLM-L6-v2` (lightweight), `all-mpnet-base-v2` (best)
- **Word2vec**: `word2vec-google-news-300` (gensim)
- **GloVe**: Available from [Stanford NLP](https://nlp.stanford.edu/projects/glove/)
- **LLMs**: Gemma, Llama, Mistral via Ollama

## The Bigger Picture

You've completed the module! You can now:

✅ **Use LLMs** for practical research tasks (summarization, extraction, analysis)
✅ **Engineer prompts** that produce reliable outputs
✅ **Extract embeddings** and use them for semantic search, clustering, and classification
✅ **Understand transformers** at an intuitive level
✅ **Apply Word2vec** for static embeddings and semantic analysis
✅ **Choose appropriate methods** (BoW, TF-IDF, embeddings, LLMs) for different tasks
✅ **Conduct complete research projects** from question to publication-ready analysis

### What's Next?

This module focused on text. The same principles extend to other modalities:

- **Module 04 (Images)**: CNNs, ResNet, Vision Transformers
- **Module 05 (Graphs)**: GNNs, spectral methods, network embeddings
- **Module 06 (LLMs)**: Advanced topics (scaling laws, emergent abilities, alignment)

The deep learning toolkit you've learned—embeddings, attention, transformers—is universal. Text, images, graphs, and multi-modal data all use similar architectures with domain-specific adaptations.

### Final Thoughts

Text is one of humanity's richest data sources. Every tweet, paper, book, and conversation is a trace of human thought, culture, and knowledge. With the tools in this module, you can:

- **Trace idea evolution** in scientific literature
- **Measure cultural shifts** in historical text
- **Analyze discourse** in online communities
- **Understand information spread** in social networks
- **Build intelligent systems** that process and generate language

The techniques you've learned are not just for NLP research—they're for understanding the complex systems of human communication, culture, and knowledge production.

**Now go forth and discover something new in the world of text.**

---

**End of Module 03**

Return to [Module Overview](overview.qmd) | Continue to [Module 04: Images →](#)